# Tests for ProcessChannel (RPC Process Module)

In [ ]:
#|default_exp rpc.test_process

In [ ]:
#|export
import pytest
import asyncio
import multiprocessing as mp
from netrun.rpc.base import ChannelClosed, RecvTimeout
from netrun.rpc.process import (
    ProcessChannel,
    SyncProcessChannel,
    create_queue_pair,
)

## Test Channel Creation

In [ ]:
#|export
@pytest.mark.asyncio
async def test_create_queue_pair():
    """Test creating a queue pair."""
    ctx = mp.get_context("spawn")
    parent_channel, child_queues = create_queue_pair(ctx)
    assert isinstance(parent_channel, ProcessChannel)
    assert isinstance(child_queues, tuple)
    assert len(child_queues) == 2

In [ ]:
await test_create_queue_pair();

In [ ]:
#|export
def test_sync_channel_creation():
    """Test creating a SyncProcessChannel."""
    ctx = mp.get_context("spawn")
    parent_channel, child_queues = create_queue_pair(ctx)
    send_q, recv_q = child_queues
    child_channel = SyncProcessChannel(send_q, recv_q)
    assert isinstance(child_channel, SyncProcessChannel)
    assert not child_channel.is_closed

In [ ]:
test_sync_channel_creation();

## Test Basic Send/Recv (Same Process)

These tests verify the channel works within the same process before
testing cross-process communication.

In [ ]:
#|export
@pytest.mark.asyncio
async def test_same_process_send_recv():
    """Test send/recv within same process using threads."""
    import threading

    ctx = mp.get_context("spawn")
    parent_channel, child_queues = create_queue_pair(ctx)
    send_q, recv_q = child_queues
    child_channel = SyncProcessChannel(send_q, recv_q)

    received = []

    def worker():
        key, data = child_channel.recv()
        received.append((key, data))
        child_channel.send("ack", data * 2)

    thread = threading.Thread(target=worker)
    thread.start()

    await parent_channel.send("test", 21)
    key, data = await parent_channel.recv(timeout=2.0)
    thread.join(timeout=2.0)

    assert received == [("test", 21)]
    assert key == "ack"
    assert data == 42

In [ ]:
await test_same_process_send_recv();

## Test Cross-Process Communication

Worker functions are imported from the workers module.

In [ ]:
#|export
from .workers import echo_worker, compute_worker

In [ ]:
#|export
@pytest.mark.asyncio
async def test_cross_process_echo():
    """Test echo communication across processes."""
    ctx = mp.get_context("spawn")
    parent_channel, child_queues = create_queue_pair(ctx)

    proc = ctx.Process(target=echo_worker, args=child_queues)
    proc.start()

    try:
        await parent_channel.send("hello", "world")
        key, data = await parent_channel.recv(timeout=5.0)

        assert key == "echo:hello"
        assert data == "world"
    finally:
        await parent_channel.close()
        proc.join(timeout=2.0)
        if proc.is_alive():
            proc.terminate()

In [ ]:
await test_cross_process_echo();

In [ ]:
#|export
@pytest.mark.asyncio
async def test_cross_process_multiple_messages():
    """Test multiple messages across processes."""
    ctx = mp.get_context("spawn")
    parent_channel, child_queues = create_queue_pair(ctx)

    proc = ctx.Process(target=echo_worker, args=child_queues)
    proc.start()

    try:
        # Send multiple messages
        for i in range(5):
            await parent_channel.send(f"msg{i}", i)

        # Receive all responses
        responses = []
        for _ in range(5):
            key, data = await parent_channel.recv(timeout=5.0)
            responses.append((key, data))

        # Check all received (order may vary)
        assert len(responses) == 5
        for key, data in responses:
            assert key.startswith("echo:msg")
    finally:
        await parent_channel.close()
        proc.join(timeout=2.0)
        if proc.is_alive():
            proc.terminate()

In [ ]:
await test_cross_process_multiple_messages();

In [ ]:
#|export
@pytest.mark.asyncio
async def test_cross_process_compute():
    """Test computation across processes."""
    ctx = mp.get_context("spawn")
    parent_channel, child_queues = create_queue_pair(ctx)

    proc = ctx.Process(target=compute_worker, args=child_queues)
    proc.start()

    try:
        await parent_channel.send("square", 7)
        key, data = await parent_channel.recv(timeout=5.0)
        assert key == "result"
        assert data == 49

        await parent_channel.send("double", 21)
        key, data = await parent_channel.recv(timeout=5.0)
        assert key == "result"
        assert data == 42
    finally:
        await parent_channel.close()
        proc.join(timeout=2.0)
        if proc.is_alive():
            proc.terminate()

In [ ]:
await test_cross_process_compute();

## Test try_recv

In [ ]:
#|export
@pytest.mark.asyncio
async def test_try_recv_empty():
    """Test try_recv on empty queue."""
    ctx = mp.get_context("spawn")
    parent_channel, _ = create_queue_pair(ctx)
    result = await parent_channel.try_recv()
    assert result is None

In [ ]:
await test_try_recv_empty();

## Test Timeout

In [ ]:
#|export
@pytest.mark.asyncio
async def test_recv_timeout():
    """Test recv timeout."""
    ctx = mp.get_context("spawn")
    parent_channel, _ = create_queue_pair(ctx)

    with pytest.raises(RecvTimeout):
        await parent_channel.recv(timeout=0.1)

In [ ]:
await test_recv_timeout();

## Test Channel Close

In [ ]:
#|export
@pytest.mark.asyncio
async def test_close():
    """Test closing channel."""
    ctx = mp.get_context("spawn")
    parent_channel, _ = create_queue_pair(ctx)

    assert not parent_channel.is_closed
    await parent_channel.close()
    assert parent_channel.is_closed

In [ ]:
await test_close();

In [ ]:
#|export
@pytest.mark.asyncio
async def test_send_on_closed_raises():
    """Test sending on closed channel raises ChannelClosed."""
    ctx = mp.get_context("spawn")
    parent_channel, _ = create_queue_pair(ctx)
    await parent_channel.close()

    with pytest.raises(ChannelClosed):
        await parent_channel.send("test", "data")

In [ ]:
await test_send_on_closed_raises();